In [1]:
from vistools.config import ConfigManager
from vistools.finding_file_helper import *
config_name = choose_file_with_stdin('configs/', 'Choose desired config file')
print("Training based on {}".format(config_name))
config_manager = ConfigManager(config_name, always_write=False)
cfg = config_manager.get_json()
import json
print(json.dumps(cfg, indent=4, sort_keys=True))


Choose desired config file
	[0]: inception_pool_config_gpu_sage.json
	[1]: inception_pool_config_sage.json
	[2]: vgg16_pool_config_sage.json
	[3]: vgg16_pool_config_gpu_sage.json
	[4]: resnet50_pool_config_sage.json
	[5]: dense_config_sage.json
	[6]: new_config_template.json
	[7]: test_config.json
	[8]: resnet50_pool_config_v1.json
Selection: 0
 Training based on inception_pool_config_gpu_sage.json
{
    "sagemaker": {
        "eval_metrics": [
            "mean_squared_error"
        ], 
        "evaluation_steps": 10, 
        "output_bucket_name": "sagemaker-us-east-1-359098845427", 
        "output_path": "s3://sagemaker-us-east-1-359098845427", 
        "train_instance_type": "ml.p2.xlarge", 
        "train_max_run": 3600, 
        "training_steps": 10000
    }, 
    "train_path": "s3://bsivisiondata/", 
    "training": {
        "hyperparameters": {
            "batch_size": 8, 
            "decay": 1e-06, 
            "learning_rate": 0.0001, 
            "nb_epochs": 1000
     

In [2]:
cat vistools/mx_sage_train.py

import os
import csv
import numpy as np
from processor import process_image
from random import shuffle
import mxnet as mx
from inceptionv3 import get_symbol


input_layer_name = 'input_1'

def get_train_context(num_cpus, num_gpus):
    if num_gpus > 0:
        return mx.gpu()
    return mx.cpu()

def get_graph():
    return get_symbol(num_outputs=9)

def train(hyperparameters, channel_input_dirs, num_cpus, num_gpus, **kwargs):
    '''
    hyperparameters: The hyperparameters passed to SageMaker TrainingJob that runs your TensorFlow training
        script. You can use this to pass hyperparameters to your training script.
    '''
    config = hyperparameters
    batch_size = config['hyperparameters']['batch_size']
    target_shape = (3,) + tuple(config['model']['target_shape'])
    
    train_image_paths, train_labels = _input_fn(channel_input_dirs['train'], 'train_data.csv', config, batch_size)
    train_imglist = zip(train_labels, train_image_paths)
    train_img_iter = mx.image.Image

In [2]:
import boto3
client = boto3.client('iam')
role_arn = client.get_role(RoleName='AmazonSageMaker-ExecutionRole-20180425T132698')['Role']['Arn']
print("Role: {}".format(role_arn))

Role: arn:aws:iam::359098845427:role/service-role/AmazonSageMaker-ExecutionRole-20180425T132698


In [3]:
output_path = cfg['sagemaker']['output_path']
instance_type = cfg['sagemaker']['train_instance_type']
train_max_run = int(cfg['sagemaker']['train_max_run'])

In [4]:
from sagemaker.mxnet import MXNet
mx_estimator = MXNet(entry_point='mx_sage_train.py',
                        role=role_arn, # AWS IAM role
                        train_instance_count=1, # number of machines to use in training (keras limited to 1)
                        train_instance_type=instance_type, #instance type
                        hyperparameters=cfg['training'], # hyperparameters passed to training funcs
                        train_max_run=train_max_run, # max training seconds before termination
                        source_dir='vistools/',
                        output_path=output_path # output bucket name
                    )


In [6]:
train_path = cfg['train_path']
mx_estimator.fit({"train": str(train_path), "test": str(train_path)})


INFO:sagemaker:Creating training-job with name: sagemaker-mxnet-2018-06-18-23-02-14-529


......................................................
2018-06-18 23:06:42,331 INFO - root - running container entrypoint
2018-06-18 23:06:42,331 INFO - root - starting train task
2018-06-18 23:06:42,351 INFO - container_support.training - Training starting
2018-06-18 23:06:44,818 INFO - mxnet_container.train - MXNetTrainingEnvironment: {'enable_cloudwatch_metrics': False, 'available_gpus': 1, 'channels': {u'test': {u'TrainingInputMode': u'File', u'RecordWrapperType': u'None', u'S3DistributionType': u'FullyReplicated'}, u'train': {u'TrainingInputMode': u'File', u'RecordWrapperType': u'None', u'S3DistributionType': u'FullyReplicated'}}, '_ps_verbose': 0, 'resource_config': {u'current_host': u'algo-1', u'network_interface_name': u'ethwe', u'hosts': [u'algo-1']}, 'user_script_name': u'mx_sage_train.py', 'input_config_dir': '/opt/ml/input/config', 'channel_dirs': {u'test': u'/opt/ml/input/data/test', u'train': u'/opt/ml/input/data/train'}, 'code_dir': '/opt/ml/code', 'output_data_dir': '/o

2018-06-18 23:07:50,990 INFO - root - Epoch[19] Train-mse=nan
2018-06-18 23:07:50,990 INFO - root - Epoch[19] Time cost=1.744
2018-06-18 23:07:51,835 INFO - root - Epoch[19] Validation-mse=nan
2018-06-18 23:07:53,589 INFO - root - Epoch[20] Train-mse=nan
2018-06-18 23:07:53,590 INFO - root - Epoch[20] Time cost=1.754
2018-06-18 23:07:54,440 INFO - root - Epoch[20] Validation-mse=nan
2018-06-18 23:07:56,171 INFO - root - Epoch[21] Train-mse=nan
2018-06-18 23:07:56,172 INFO - root - Epoch[21] Time cost=1.732
2018-06-18 23:07:57,028 INFO - root - Epoch[21] Validation-mse=nan
2018-06-18 23:07:58,777 INFO - root - Epoch[22] Train-mse=nan
2018-06-18 23:07:58,778 INFO - root - Epoch[22] Time cost=1.749
2018-06-18 23:07:59,651 INFO - root - Epoch[22] Validation-mse=nan
2018-06-18 23:08:01,400 INFO - root - Epoch[23] Train-mse=nan
2018-06-18 23:08:01,401 INFO - root - Epoch[23] Time cost=1.750
2018-06-18 23:08:02,259 INFO - root - Epoch[23] Validation-mse=nan
2018-06-18 23:08:04,019 INFO - root

2018-06-18 23:09:29,806 INFO - root - Epoch[57] Train-mse=nan
2018-06-18 23:09:29,807 INFO - root - Epoch[57] Time cost=1.751
2018-06-18 23:09:30,650 INFO - root - Epoch[57] Validation-mse=nan
2018-06-18 23:09:32,400 INFO - root - Epoch[58] Train-mse=nan
2018-06-18 23:09:32,401 INFO - root - Epoch[58] Time cost=1.751
2018-06-18 23:09:33,266 INFO - root - Epoch[58] Validation-mse=nan
2018-06-18 23:09:35,014 INFO - root - Epoch[59] Train-mse=nan
2018-06-18 23:09:35,015 INFO - root - Epoch[59] Time cost=1.748
2018-06-18 23:09:35,869 INFO - root - Epoch[59] Validation-mse=nan
2018-06-18 23:09:37,621 INFO - root - Epoch[60] Train-mse=nan
2018-06-18 23:09:37,622 INFO - root - Epoch[60] Time cost=1.752
2018-06-18 23:09:38,451 INFO - root - Epoch[60] Validation-mse=nan
2018-06-18 23:09:40,213 INFO - root - Epoch[61] Train-mse=nan
2018-06-18 23:09:40,214 INFO - root - Epoch[61] Time cost=1.763
2018-06-18 23:09:41,048 INFO - root - Epoch[61] Validation-mse=nan
2018-06-18 23:09:42,797 INFO - root

2018-06-18 23:11:08,513 INFO - root - Epoch[95] Train-mse=nan
2018-06-18 23:11:08,514 INFO - root - Epoch[95] Time cost=1.756
2018-06-18 23:11:09,372 INFO - root - Epoch[95] Validation-mse=nan
2018-06-18 23:11:11,133 INFO - root - Epoch[96] Train-mse=nan
2018-06-18 23:11:11,134 INFO - root - Epoch[96] Time cost=1.761
2018-06-18 23:11:11,998 INFO - root - Epoch[96] Validation-mse=nan
2018-06-18 23:11:13,761 INFO - root - Epoch[97] Train-mse=nan
2018-06-18 23:11:13,762 INFO - root - Epoch[97] Time cost=1.764
2018-06-18 23:11:14,645 INFO - root - Epoch[97] Validation-mse=nan
2018-06-18 23:11:16,390 INFO - root - Epoch[98] Train-mse=nan
2018-06-18 23:11:16,391 INFO - root - Epoch[98] Time cost=1.746
2018-06-18 23:11:17,248 INFO - root - Epoch[98] Validation-mse=nan
2018-06-18 23:11:18,995 INFO - root - Epoch[99] Train-mse=nan
2018-06-18 23:11:18,997 INFO - root - Epoch[99] Time cost=1.749
2018-06-18 23:11:19,847 INFO - root - Epoch[99] Validation-mse=nan
2018-06-18 23:11:21,591 INFO - root

...

2018-06-18 23:44:13,682 INFO - root - Epoch[855] Train-mse=nan
2018-06-18 23:44:13,683 INFO - root - Epoch[855] Time cost=1.753
2018-06-18 23:44:14,531 INFO - root - Epoch[855] Validation-mse=nan
2018-06-18 23:44:16,277 INFO - root - Epoch[856] Train-mse=nan
2018-06-18 23:44:16,277 INFO - root - Epoch[856] Time cost=1.746
2018-06-18 23:44:17,111 INFO - root - Epoch[856] Validation-mse=nan
2018-06-18 23:44:18,857 INFO - root - Epoch[857] Train-mse=nan
2018-06-18 23:44:18,858 INFO - root - Epoch[857] Time cost=1.747
2018-06-18 23:44:19,702 INFO - root - Epoch[857] Validation-mse=nan
2018-06-18 23:44:21,445 INFO - root - Epoch[858] Train-mse=nan
2018-06-18 23:44:21,446 INFO - root - Epoch[858] Time cost=1.743
2018-06-18 23:44:22,301 INFO - root - Epoch[858] Validation-mse=nan
2018-06-18 23:44:24,067 INFO - root - Epoch[859] Train-mse=nan
2018-06-18 23:44:24,069 INFO - root - Epoch[859] Time cost=1.767
2018-06-18 23:44:24,950 INFO - root - Epoch[859] Validation-mse=nan
2018-06-18 23:44:26,

2018-06-18 23:45:52,882 INFO - root - Epoch[893] Train-mse=nan
2018-06-18 23:45:52,883 INFO - root - Epoch[893] Time cost=1.744
2018-06-18 23:45:53,732 INFO - root - Epoch[893] Validation-mse=nan
2018-06-18 23:45:55,495 INFO - root - Epoch[894] Train-mse=nan
2018-06-18 23:45:55,496 INFO - root - Epoch[894] Time cost=1.764
2018-06-18 23:45:56,332 INFO - root - Epoch[894] Validation-mse=nan
2018-06-18 23:45:58,081 INFO - root - Epoch[895] Train-mse=nan
2018-06-18 23:45:58,082 INFO - root - Epoch[895] Time cost=1.750
2018-06-18 23:45:58,972 INFO - root - Epoch[895] Validation-mse=nan
2018-06-18 23:46:00,717 INFO - root - Epoch[896] Train-mse=nan
2018-06-18 23:46:00,719 INFO - root - Epoch[896] Time cost=1.747
2018-06-18 23:46:01,599 INFO - root - Epoch[896] Validation-mse=nan
2018-06-18 23:46:03,353 INFO - root - Epoch[897] Train-mse=nan
2018-06-18 23:46:03,354 INFO - root - Epoch[897] Time cost=1.755
2018-06-18 23:46:04,211 INFO - root - Epoch[897] Validation-mse=nan
2018-06-18 23:46:05,

2018-06-18 23:47:32,334 INFO - root - Epoch[931] Train-mse=nan
2018-06-18 23:47:32,335 INFO - root - Epoch[931] Time cost=1.767
2018-06-18 23:47:33,204 INFO - root - Epoch[931] Validation-mse=nan
2018-06-18 23:47:34,984 INFO - root - Epoch[932] Train-mse=nan
2018-06-18 23:47:34,985 INFO - root - Epoch[932] Time cost=1.781
2018-06-18 23:47:35,855 INFO - root - Epoch[932] Validation-mse=nan
2018-06-18 23:47:37,606 INFO - root - Epoch[933] Train-mse=nan
2018-06-18 23:47:37,608 INFO - root - Epoch[933] Time cost=1.752
2018-06-18 23:47:38,467 INFO - root - Epoch[933] Validation-mse=nan
2018-06-18 23:47:40,215 INFO - root - Epoch[934] Train-mse=nan
2018-06-18 23:47:40,216 INFO - root - Epoch[934] Time cost=1.749
2018-06-18 23:47:41,075 INFO - root - Epoch[934] Validation-mse=nan
2018-06-18 23:47:42,828 INFO - root - Epoch[935] Train-mse=nan
2018-06-18 23:47:42,829 INFO - root - Epoch[935] Time cost=1.754
2018-06-18 23:47:43,700 INFO - root - Epoch[935] Validation-mse=nan
2018-06-18 23:47:45,

2018-06-18 23:49:06,448 INFO - root - Epoch[967] Train-mse=nan
2018-06-18 23:49:06,449 INFO - root - Epoch[967] Time cost=1.757
2018-06-18 23:49:07,307 INFO - root - Epoch[967] Validation-mse=nan
2018-06-18 23:49:09,080 INFO - root - Epoch[968] Train-mse=nan
2018-06-18 23:49:09,081 INFO - root - Epoch[968] Time cost=1.774
2018-06-18 23:49:09,959 INFO - root - Epoch[968] Validation-mse=nan
2018-06-18 23:49:11,724 INFO - root - Epoch[969] Train-mse=nan
2018-06-18 23:49:11,725 INFO - root - Epoch[969] Time cost=1.766
2018-06-18 23:49:12,592 INFO - root - Epoch[969] Validation-mse=nan
2018-06-18 23:49:14,343 INFO - root - Epoch[970] Train-mse=nan
2018-06-18 23:49:14,344 INFO - root - Epoch[970] Time cost=1.753
2018-06-18 23:49:15,208 INFO - root - Epoch[970] Validation-mse=nan
2018-06-18 23:49:16,974 INFO - root - Epoch[971] Train-mse=nan
2018-06-18 23:49:16,975 INFO - root - Epoch[971] Time cost=1.766
2018-06-18 23:49:17,842 INFO - root - Epoch[971] Validation-mse=nan
2018-06-18 23:49:19,

ValueError: Error training sagemaker-mxnet-2018-06-18-23-02-14-529: Failed Reason: AlgorithmError: uncaught exception during training: global name 'mod' is not defined
Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/container_support/training.py", line 36, in start
    fw.train()
  File "/usr/local/lib/python2.7/dist-packages/mxnet_container/train.py", line 191, in train
    model = user_module.train(**kwargs_to_pass)
  File "/opt/ml/code/mx_sage_train.py", line 55, in train
    return mod
NameError: global name 'mod' is not defined

